In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import os
import pandas as pd
from tifffile import imread, memmap

from caImageAnalysis import BrukerFish
from caImageAnalysis.utils import rotate_image
from caImageAnalysis.visualize import visualize_images, visualize_volumes

### Change the exp_folder to the path of the experiment

In [108]:
# Change this path to the experiment folder
exp_folder = r'/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_glucose_25mM_21_20240109'
region = ''
remove_pulses = None
gavage = True

In [109]:
# Initialize BrukerFish
fish = BrukerFish(exp_folder, region=region, remove_pulses=remove_pulses, gavage=gavage)
fish.data_paths

Batch exists. Loading batch.pickle


/Users/minelarinel/caImageAnalysis/caImageAnalysis/bruker_2P/bruker.py:178: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 178 of the file /Users/minelarinel/caImageAnalysis/caImageAnalysis/bruker_2P/bruker.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  log = file.read()
/Users/minelarinel/mambaforge/envs/mescore/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be 

{'raw': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_glucose_25mM_21_20240109/vagal_R-000'),
 'frametimes': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_glucose_25mM_21_20240109/vagal_R-000_frametimes.txt'),
 'raw_image': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_glucose_25mM_21_20240109/vagal_R-000_ch2.tif'),
 'mesmerize': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_glucose_25mM_21_20240109/mesmerize-batch'),
 'temporal': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_glucose_25mM_21_20240109/temporal.h5'),
 'vol_temporal': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_glucose_25mM_21_20240109/vol_temporal.pkl'),
 'clusters': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_glucose_25mM_21_20240109/clusters.pkl'),
 'unrolled_temporal': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_glucose_25mM_21_20240109/unrolled_temporal.

In [110]:
# If running for the first time, create a combined frametimes.txt and combined tif files
fish.create_frametimes_txt()
# fish.combine_channel_images('Ch2')

/Users/minelarinel/caImageAnalysis/caImageAnalysis/bruker_2P/bruker.py:208: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 208 of the file /Users/minelarinel/caImageAnalysis/caImageAnalysis/bruker_2P/bruker.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  log = file.read()


### If you have any phase offset problems, stop here and run the 0.5_fix_phase.ipynb

In [6]:
# Check the combined image. If needed, rotate the raw image by angle.
if 'markpoints' in fish.data_paths.keys() or 'vagal_R' in fish.data_paths['raw'].name:
    # coordinate conversion is a pain, rotate the image later
    # and vagal_R is collected in the correct orientation
    pass
else:
    if 'vagal_L' in fish.data_paths['raw'].name:
        angle = 180
    else:
        angle = 90

    fish.rotate_image(fish.data_paths['raw_image'], angle=angle)

In [7]:
# When the rotation angle is accurate, also rotate and create the anatomy.tif stack
if 'markpoints' in fish.data_paths.keys() or 'vagal_R' in fish.data_paths['raw'].name:
    # coordinate conversion is a pain, rotate the image later
    # and vagal_R is collected in the correct orientation
    pass
else:
    anatomy_img = imread(fish.data_paths['anatomy'])
    anatomy_img_rotated = rotate_image(anatomy_img, path=fish.exp_path.joinpath('anatomy.tif'), angle=angle)
    fish.process_filestructure()

    plt.imshow(anatomy_img_rotated[0])

In [111]:
# Split the raw volumetric image to individual planes
# If it's not volumetric, creates a frametimes.h5 file in the exp_path
if fish.volumetric:
    fish.split_bruker_volumes('Ch2')
else:
    frametimes = fish.frametimes_df.copy()
    frametimes.reset_index(drop=True)
    if len(fish.region) > 0:
        frametimes.to_hdf(os.path.join(fish.exp_path, f'{region}_frametimes.h5'), 'frametimes')
    else:
        frametimes.to_hdf(os.path.join(fish.exp_path, f'frametimes.h5'), 'frametimes')
    fish.process_filestructure()

/Users/minelarinel/caImageAnalysis/caImageAnalysis/bruker_2P/bruker.py:325: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->time,key->block0_values] [items->Index(['time'], dtype='object')]

  


Batch exists. Loading batch.pickle


In [112]:
fish.frametimes_df

,time,pulse
0,14:43:15,0
1,14:43:15.153384,0
2,14:43:15.306768,0
3,14:43:15.460152,0
4,14:43:15.613535,0
...,...,...
5865,14:58:14.596386,4
5866,14:58:14.074977,4
5867,14:58:14.903154,4
5868,14:58:15.056537,4


## Visualize plane(s)

In [10]:
if fish.volumetric:
    iw = visualize_volumes(fish)
else:
    img = memmap(fish.data_paths['raw_image'])
    iw = visualize_images(imgs=img)

iw.show()

RFBOutputContext()

No such comm: 18378026f82c4ff2a530f226587b0a1e
No such comm: 18378026f82c4ff2a530f226587b0a1e


In [11]:
iw.plot.canvas.close()